<a href="https://colab.research.google.com/github/SelimOzn/ml/blob/main/EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
X,y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
voting_clf=VotingClassifier(estimators=[
 ('lr', LogisticRegression(random_state=42)),
 ('rf', RandomForestClassifier(random_state=42)),
 ('svc', SVC(random_state=42))

])
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.912

In [ ]:
for name,clf in voting_clf.named_estimators_.items():  #named_estimators hali fit olmamış modelleri içerir.
  print(name, clf.score(X_test, y_test))

lr 0.864
rf 0.896
svc 0.896


In [ ]:
voting_clf.predict(X_test[:1])

array([1])

In [ ]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_] #Named olmadan direkt estimatorları döndürür. Named isimlerini de döndürür.

[array([1]), array([1]), array([0])]

In [ ]:
voting_clf.voting="soft"
voting_clf.named_estimators["svc"].probability=True
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.92

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples = 100, #500=Tree sayısı, 100=Subsetin instances sayısı,
                                                                                              #n_jobs=kullanılacak işlemci sayısı,-1 hepsi
                            n_jobs=-1, random_state=42)


In [ ]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, oob_score=True, n_jobs=-1, random_state=42) #Test sette 0.896lık accuracy, OOB'ye göre
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.896

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)


0.92

In [ ]:
bag_clf.oob_decision_function_[:3] #Her instance için OOB decision func. Her training instance için class proba verir.

array([[0.32352941, 0.67647059],
       [0.3375    , 0.6625    ],
       [1.        , 0.        ]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(random_state=42, n_jobs=-1, max_leaf_nodes=16, n_estimators=500)

In [ ]:
rnd_clf.fit(X_train,y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [ ]:
y_pred = rnd_clf.predict(X_test)

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16), n_estimators=500, random_state=42)
#max_feature diyince random feature subset oluşturur. Split yaparken o featurelar arasından en iyisine göre split kontrolünü yapar.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris.data, iris.target)

RandomForestClassifier(n_estimators=500, random_state=42)

In [ ]:
for score, name in zip(rnd_clf.feature_importances_, iris.data.columns):
  print(round(score,2), name)

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [ ]:
np.random.seed(42)
X = np.random.rand(100,1) - 0.5
y = 3*X[:,0] ** 2 + 0.05 * np.random.rand(100) #y=3x^2 + Gaussian Noise

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X,y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [ ]:
y2 = y - tree_reg.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [ ]:
y3 = y2-tree_reg2.predict(X)

In [ ]:
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [ ]:
X_new = np.array([[-0.4], [0.], [0.5]])
sum(tree.predict(X_new) for tree in (tree_reg, tree_reg2, tree_reg3))


array([0.50199918, 0.08055604, 0.74072131])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

In [ ]:
gbrt_best = GradientBoostingRegressor(max_depth=2, learning_rate=0.05, n_estimators=500, n_iter_no_change=10, random_state=42)

In [ ]:
gbrt_best.fit(X,y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

In [ ]:
gbrt_best.n_estimators_

86

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

hgb_reg = make_pipeline(make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
                                              remainder="passthrough"), HistGradientBoostingRegressor(categorical_features=[0], random_state=42))

#categorical_features = kategorik columnun indeces olmalı

In [ ]:
from sklearn.ensemble import StackingClassifier
stacking_clf = StackingClassifier(estimators = [("lr", LogisticRegression(random_state=42)),
("rf", RandomForestClassifier(random_state=42)),
("svc", SVC(probability=True, random_state=42))
],
                                  final_estimator = RandomForestClassifier(random_state=43),  #final_estimator tanımlanmazsa StackingClassifier log.reg; Regressor RidgeCV use.
                                  cv = 5)


In [ ]:
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))